In [1]:
from sphere import Sphere,mat

In [2]:
import numpy as np
def J_z(d):
    j = (d-1)/2
    return np.diag([j-i for i in range(d)])
def r(m, j):
    return np.sqrt(j*(j+1)-m*(m+1))

def J_p(d):
    j = (d-1)/2
    return np.diag([r(j-1-i,j) for i in range(d-1)], k=1)
    
def J_m(d):
    return J_p(d).T

def J_x(d):
    return (J_p(d)+J_m(d))/2
def J_y(d):
    return (J_p(d)-J_m(d))/(2j)


In [6]:
def mat3(theta, phi,n=2):
    return np.sin(theta)*np.cos(phi)*J_x(n)+\
           np.sin(theta)*np.sin(phi)*J_y(n)+\
           np.cos(theta)*J_z(n)
            

In [20]:
s = .4999
s = .5001
s = .7
def mat2(theta, phi):
    return (1-s)*mat(0,0) + s*mat(theta, phi)

In [21]:
sphere = Sphere(delta_phi=.4,delta_theta=.04, mat = mat2)
sphere.chern_numbers()

[-1, 1]